<a href="https://colab.research.google.com/github/SophiaHe/UFOSightingData_NLP_project/blob/main/UFOSighting_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!ls "./gdrive/MyDrive/Data Science Case"

nuforc_reports.csv  README-updated.md


In [3]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta 
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
import seaborn as sns
import pickle
from matplotlib import pyplot
import plotly.express as px
import math
import itertools

In [28]:
path = "./gdrive/MyDrive/Data Science Case/"

df = pd.read_csv(path+"nuforc_reports.csv")


In [5]:
print(df.info())
print(df.shape)
print(df.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88125 entries, 0 to 88124
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   summary         88095 non-null  object 
 1   city            87891 non-null  object 
 2   state           82890 non-null  object 
 3   date_time       86938 non-null  object 
 4   shape           85627 non-null  object 
 5   duration        84954 non-null  object 
 6   stats           88088 non-null  object 
 7   report_link     88125 non-null  object 
 8   text            88070 non-null  object 
 9   posted          86938 non-null  object 
 10  city_latitude   72013 non-null  float64
 11  city_longitude  72013 non-null  float64
dtypes: float64(2), object(10)
memory usage: 8.1+ MB


,summary,city,state,date_time,shape,duration,stats,report_link,text,posted,city_latitude,city_longitude
0,My wife was driving southeast on a fairly popu...,Chester,VA,2019-12-12T18:43:00,light,5 seconds,Occurred : 12/12/2019 18:43 (Entered as : 12/...,http://www.nuforc.org/webreports/151/S151739.html,My wife was driving southeast on a fairly popu...,2019-12-22T00:00:00,37.343152,-77.408582
1,I think that I may caught a UFO on the NBC Nig...,Rocky Hill,CT,2019-03-22T18:30:00,circle,3-5 seconds,Occurred : 3/22/2019 18:30 (Entered as : 03/2...,http://www.nuforc.org/webreports/145/S145297.html,I think that I may caught a UFO on the NBC Nig...,2019-03-29T00:00:00,41.664800,-72.639300
2,I woke up late in the afternoon 3:30-4pm. I we...,NaN,NaN,NaN,NaN,NaN,Occurred : 4/1/2019 15:45 (Entered as : April...,http://www.nuforc.org/webreports/145/S145556.html,I woke up late in the afternoon 3:30-4pm. I w...,NaN,NaN,NaN
3,I was driving towards the intersection of fall...,Ottawa,ON,2019-04-17T02:00:00,teardrop,10 seconds,Occurred : 4/17/2019 02:00 (Entered as : 04-1...,http://www.nuforc.org/webreports/145/S145697.html,I was driving towards the intersection of fall...,2019-04-18T00:00:00,45.381383,-75.708501
4,"In Peoria Arizona, I saw a cigar shaped craft ...",Peoria,NY,2009-03-15T18:00:00,cigar,2 minutes,Occurred : 3/15/2009 18:00 (Entered as : 03/1...,http://www.nuforc.org/webreports/145/S145723.html,"In Peoria, Arizona, I saw a cigar shaped craft...",2019-04-18T00:00:00,NaN,NaN


# **Clean the data**

## convert date/time into a standard format

In [29]:
df['date_time'] = pd.to_datetime(df['date_time'])
df['date_time'].value_counts()

2015-11-07 18:00:00    104
2014-07-04 22:00:00     46
2010-07-04 22:00:00     37
2012-07-04 22:00:00     34
2015-07-04 22:00:00     27
                      ... 
2015-04-26 22:30:00      1
2014-08-14 20:50:00      1
2006-12-31 19:45:00      1
2013-06-15 20:50:00      1
2008-04-13 22:30:00      1
Name: date_time, Length: 73122, dtype: int64

## convert durations to an interval in seconds

For this data processing step, I separated the duration unit from the numbers and convert the duration into seconds. Note that there are special character involved in the duration column and there might be "incorrect" transformation involved. For instance, "3-5 seconds" got transformed into 35 seconds. There are a total of 13457 rows that contains special characters in the duration column. Given the time limit of this assignment, I removed those observations to simplify the data processing step. Furthermore, there are many different variation of the duration units within the duration column (e.g. "3 seconds", "3 hrs","all night" etc.). To preserve the most amount of data, I only included rows that have duration units which contains those strings: "hour","hr","min","sec","day". For example, "3 mins" and "3 minutes" will both be preserved. 

After those data processing, there is a total of **66292 rows** in the dataset. Lastly, I converted all duration into seconds by creating a column called "duration_num_second".

In [30]:
df = df[df.duration.str.contains(r'[@#&$%+-/*]',na=False) == False]

In [53]:
df['duration'] = df['duration'].str.lower()
df['duration_unit'] = df['duration'].str.findall(r'[A-Za-z]').str.join('') # obtain the time unit from duration column
df['duration_num'] = df['duration'].str.findall(r'\d').str.join('') # obtain the digits before the duration unit
df['duration_num'] = np.where(df['duration_num'].str.isdigit() == False,0,df['duration_num']) 
df['duration_num']= pd.to_numeric(df['duration_num'], errors='coerce')
print(df['duration_num'].value_counts())

5.0       10271
10.0       8041
2.0        7650
1.0        7595
3.0        5815
          ...  
355.0         1
2235.0        1
63.0          1
1628.0        1
255.0         1
Name: duration_num, Length: 451, dtype: int64


In [54]:
search = ["hour","hr","min","sec","day"]
df[df.duration_unit.str.contains('|'.join(search),na=False) == True].shape

(66292, 15)

In [ ]:
# durations = df['duration'].value_counts().rename_axis('unique_values').reset_index(name='counts')
# durations[durations['counts']>100]
# durations[durations['unique_values'].str.contains("minutes")]
# durations['unique_values2']= durations['unique_values'].str.lower().str.findall(r'[A-Za-z]').str.join('')
# durations['unique_values2'].value_counts().head(55)

In [55]:
# common unique duration units after filtering out rows that contains special characters
print(df['duration_unit'].value_counts().head(30))

minutes           31261
seconds           16283
minute             4300
min                3267
                   2269
hour               1760
hours              1696
mins               1259
sec                 970
second              642
unknown             470
ongoing             298
secs                207
tominutes           200
fewseconds          175
toseconds           159
fewminutes          146
aboutminutes        143
hrs                 132
afewseconds         113
severalminutes      106
aboutseconds         98
allnight             92
oneminute            87
notsure              75
days                 74
hr                   69
afewminutes          66
tomin                61
severalhours         60
Name: duration_unit, dtype: int64


In [65]:
# convert minutes, hours, days into seconds
df['duration_num_second'] = np.where(df['duration_unit'].str.contains("min", na=False) == True,
                                     df['duration_num']*60,
                                     np.where(df['duration_unit'].str.contains('|'.join(["hour","hr"]),na=False) == True,
                                              df['duration_num']*3600,
                                              np.where(df['duration_unit'].str.contains("day", na=False) == True,
                                                       df['duration_num']*3600*24,
                                                       df['duration_num'])
                                             )
                                     )
df[['duration','duration_num','duration_num_second','duration_unit']].tail(5)


,duration,duration_num,duration_num_second,duration_unit
88120,3 minutes,3.0,180.0,minutes
88121,20 seconds,20.0,20.0,seconds
88122,20 seconds,20.0,20.0,seconds
88123,2 minutes,2.0,120.0,minutes
88124,3 minutes,3.0,180.0,minutes


##add a country field and store the country in ISO 3166-1

In [66]:
us_states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
ca_provs = ['AB','BC','MB','NB','NL','NS','NT','NU','ON','PE','QC','SK','YT']

In [67]:
conditions = [
    (df["state"].isin(us_states)),
    (df["state"].isin(ca_provs)),
    (~df["state"].isin(us_states) & (~df["state"].isin(ca_provs)))
    ]
values = ['USA', 'CAN', 'OTHER']

df['Country'] = np.select(conditions, values)

In [68]:
df["Country"].value_counts().rename_axis('unique_values').reset_index(name='counts')

,unique_values,counts
0,USA,67604
1,OTHER,4551
2,CAN,2513


## Process missing values

In [69]:
df['summary'] = np.where(df['summary'].isnull() == True, 'Unknown',df['summary'])
df['city'] = np.where(df['city'].isnull() == True, 'Unknown',df['city'])
df['state'] = np.where(df['state'].isnull() == True, 'Unknown',df['state'])
df['text'] = np.where(df['text'].isnull() == True, 'Unknown',df['text'])
df['shape'] = np.where(df['shape'].isnull() == True, 'Unknown',df['shape'])

## derive more time-related variables

In [70]:
df['year'], df['month'],df['dayofweek'],df['hour'] = df['date_time'].dt.year, df['date_time'].dt.month, df['date_time'].dt.dayofweek+1, df['date_time'].dt.hour
conditions = [
  ((df['hour'] >= 0) & (df['hour'] < 6)),
  ((df['hour'] >= 6) & (df['hour'] <= 12)),
  ((df['hour'] > 12) & (df['hour'] <= 18)),
  (df['hour'] > 18),
   (df['hour'].isnull() == True)
]
values = ['night','morning','afternoon','evening','unknown']
df['time_of_day'] = np.select(conditions,values)
df.head(5)

,summary,city,state,date_time,shape,duration,stats,report_link,text,posted,city_latitude,city_longitude,duration_unit,duration_num,duration_num2,duration_num_second,Country,year,month,dayofweek,hour,time_of_day
0,My wife was driving southeast on a fairly popu...,Chester,VA,2019-12-12 18:43:00,light,5 seconds,Occurred : 12/12/2019 18:43 (Entered as : 12/...,http://www.nuforc.org/webreports/151/S151739.html,My wife was driving southeast on a fairly popu...,2019-12-22T00:00:00,37.343152,-77.408582,seconds,5.0,5.0,5.0,USA,2019.0,12.0,4.0,18.0,afternoon
2,I woke up late in the afternoon 3:30-4pm. I we...,Unknown,Unknown,NaT,Unknown,NaN,Occurred : 4/1/2019 15:45 (Entered as : April...,http://www.nuforc.org/webreports/145/S145556.html,I woke up late in the afternoon 3:30-4pm. I w...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OTHER,NaN,NaN,NaN,NaN,unknown
3,I was driving towards the intersection of fall...,Ottawa,ON,2019-04-17 02:00:00,teardrop,10 seconds,Occurred : 4/17/2019 02:00 (Entered as : 04-1...,http://www.nuforc.org/webreports/145/S145697.html,I was driving towards the intersection of fall...,2019-04-18T00:00:00,45.381383,-75.708501,seconds,10.0,10.0,10.0,CAN,2019.0,4.0,3.0,2.0,night
4,"In Peoria Arizona, I saw a cigar shaped craft ...",Peoria,NY,2009-03-15 18:00:00,cigar,2 minutes,Occurred : 3/15/2009 18:00 (Entered as : 03/1...,http://www.nuforc.org/webreports/145/S145723.html,"In Peoria, Arizona, I saw a cigar shaped craft...",2019-04-18T00:00:00,NaN,NaN,minutes,2.0,120.0,120.0,USA,2009.0,3.0,7.0,18.0,afternoon
5,"The object has flashing lights that are green,...",Kirbyville,TX,2019-04-02 20:25:00,disk,15 minutes,Occurred : 4/2/2019 20:25 (Entered as : 04/02...,http://www.nuforc.org/webreports/145/S145476.html,"The object has flashing lights that are green,...",2019-04-08T00:00:00,30.677200,-94.005200,minutes,15.0,900.0,900.0,USA,2019.0,4.0,2.0,20.0,evening


# EDA & Data Analysis

##  bucket the data by date, shape, or location. What do you observe? Why? can you interpret your findings?

This datasets contains UFO sighting reports from as early as 1969 to 2019, covering over 88K incidents that are reported to the police. The specific date & time, detailed description of the events, as well as locations for the sightings are recorded. 

Based on the states recorded in the data, majority of the events are reported within the US (90.6%). Additionally, majority of the UFO sighting events occurred during the evening (55.3%), which is defined between 6pm to 12am. Intuitively, since it's most likely dark outside when people reported those events, some commonly known objects can be more easily misunderstood to be UFO. Additionally, during the day, people are less likely to observe unusually bright objects in the sky. As illustrated below, the most commonly reported shapes are "bright" and "circular", which can be more easily observed during dark hours. Within the data, more recent sighting events are reported (2012 - 2015). The reasons behind such observation can be quite diverse. More specifically, the months when UFO sighting events are mostly reported are during the summer months (July, August & September), where people in the US and Canada are generally more active and tend to spend more time outdoor. Furthermore, the clearer weather are more prevalent during those months and any abnormal obejcts can be more easily observed during that time. Weekends including Fridays are the most commonly days of the week when UFO sightings are reported. New York, Phonix and Las Vagas are the top 3 cities where most of events are reported. 

In [ ]:
time_of_day = df['time_of_day'].value_counts().rename_axis("Time_of_Day").reset_index(name = "counts")
fig = px.pie(time_of_day, values='counts', names='Time_of_Day', title='Distribution of Time of the Day for UFO Sighting Events')
fig.show()

In [ ]:
year = df['year'].value_counts().rename_axis("year").reset_index(name = "counts").head(10)
fig = px.pie(year, values='counts', names='year', title='Distribution of year for UFO Sighting Events')
fig.show()

In [ ]:
month = df['month'].value_counts().rename_axis("month").reset_index(name = "counts")
fig = px.pie(month, values='counts', names='month', title='Distribution of month for UFO Sighting Events')
fig.show()

In [ ]:
dayofweek = df['dayofweek'].value_counts().rename_axis("dayofweek").reset_index(name = "counts")
fig = px.pie(dayofweek, values='counts', names='dayofweek', title='Distribution of day of the week for UFO Sighting Events')
fig.show()

In [ ]:
shape = df['shape'].value_counts().rename_axis("shape").reset_index(name = "counts")
fig = px.pie(shape, values='counts', names='shape', title='Distribution of shape for UFO Sighting Events')
fig.show()

In [ ]:
Country = df['Country'].value_counts().rename_axis("Country").reset_index(name = "counts")
fig = px.pie(Country, values='counts', names='Country', title='Distribution of Countrie for UFO Sighting Events')
fig.show()

In [ ]:
city = df[df['Country'] == "USA"]['city'].value_counts().rename_axis("city").reset_index(name = "counts").head(10)
fig = px.pie(city, values='counts', names='city', title='Distribution of US cities for UFO Sighting Events')
fig.show()

In [ ]:
state = df[df['Country'] == "USA"]['state'].value_counts().rename_axis("state").reset_index(name = "counts").head(10)
fig = px.pie(state, values='counts', names='state', title='Distribution of US states for UFO Sighting Events')
fig.show()

## what trends can you identify in the sightings in each location? How is the trend across different location within a short time period? Why?

As we see from the line chart below, there is a significant increase in number of UFO sighting reports between 2005 and 2008 & between 2010 and 2014. 

In [23]:
# report volumn over the years
data = df.groupby('year')['year'].count().rename_axis('year').reset_index(name='counts')
fig = px.line(data, x="year", y="counts", title='Numer of UFO Sighting Reports from 1969 to 2019')
fig.show()

Similar trend can be observed for total number of sighting report in the US, since the dataset is made up with majority of the incidents reported in the US.

In [25]:
# report volumn in the US over the years
data = df[df['Country'] == 'USA'].groupby('year')['year'].count().rename_axis('year').reset_index(name='counts')
fig = px.line(data, x="year", y="counts", title='Numer of UFO Sighting Reports in the US from 1969 to 2019')
fig.show()

As we can see from 2 charts below, the summer months (July, August & September) as well as the weekends (Friday, Saturday & Sunday) are driving up the total number of incidents reported over the years.

In [35]:
# trend in report volumn by month over the years 
data = df.groupby(['year','month']).size().to_frame(name = 'counts').reset_index()
conditions = [
  data['month'] == 1,
  data['month'] == 2,
  data['month'] == 3,
  data['month'] == 4,
  data['month'] == 5,
  data['month'] == 6,
  data['month'] == 7,
  data['month'] == 8,
  data['month'] == 9,
  data['month'] == 10,
  data['month'] == 11,
  data['month'] == 12
]
values = ['January',"February",'March','April','May','June','July','August','September','October','November','December']
data['month'] = np.select(conditions,values)
fig = px.line(data, x="year", y="counts", color='month',title='Numer of UFO Sighting Reports from 1969 to 2019 By Month')
fig.show()

In [36]:
# trend in report volumn by day of the week over the years 
data = df.groupby(['year','dayofweek']).size().to_frame(name = 'counts').reset_index()
conditions = [
  data['dayofweek'] == 1,
  data['dayofweek'] == 2,
  data['dayofweek'] == 3,
  data['dayofweek'] == 4,
  data['dayofweek'] == 5,
  data['dayofweek'] == 6,
  data['dayofweek'] == 7
]
values = ['Monday',"Tuesday",'Wednesday','Thursday','Friday','Saturday','Sunday']
data['dayofweek'] = np.select(conditions,values)
fig = px.line(data, x="year", y="counts", color='dayofweek',title='Numer of UFO Sighting Reports from 1969 to 2019 By Day of The Week')
fig.show()

Similarly, during the years when we observed siginificant increase in number of sighting reports, we can also see sharpe increase in number of incidents occurred in the evening (6pm to 12am).

In [37]:
# trend in report volumn by time of the day over the years 
data = df.groupby(['year','time_of_day']).size().to_frame(name = 'counts').reset_index()
# conditions = [
#   data['time_of_day'] == 1,
#   data['time_of_day'] == 2,
#   data['time_of_day'] == 3,
#   data['time_of_day'] == 4
# ]
# values = ['Monday',"Tuesday",'Wednesday','Thursday','Friday','Saturday','Sunday']
# data['dayofweek'] = np.select(conditions,values)
fig = px.line(data, x="year", y="counts", color='time_of_day',title='Numer of UFO Sighting Reports from 1969 to 2019 By Time of the Day')
fig.show()

## NLP: using LDA to analyze the text column

In [ ]:
df['summary'][0]

'My wife was driving southeast on a fairly populated main side road, it was dark out side at about 6:43pm, And my wife exclaimed” fallin'

In [23]:
df['text'][0]

'My wife was driving southeast on a fairly populated main side road, it was dark out side at about 6:43pm, And my wife exclaimed” falling star baby look quick!” When I looked up I saw not a falling star but a bright ball of light , one that was closer than any shooting star I have ever seen, it had a blue glow  changing into green colors of light as it fell from the sky. It fell as if falling from an invisible opening in the sky... the night was a crystal clear night sky  so no clouds or precipitation to obstruct our view and the object was closer than any I have ever witnessed before. The way the object fell was too slow to be a meteor or falling star, also noting that there was no “light trail” following behind the object as it fell. We watched it fall for about 5 seconds before it disappeared into the dark night sky somewhere close to the earth surface.  It was the strangest light; object; ufo... I have encountered and felt a duty to report it.'

In [76]:
# Run in python console
import nltk; 
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [77]:
import gensim
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
porter = PorterStemmer()
lancaster=LancasterStemmer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text, deacc=True):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_text(token))
            
    return result



In [26]:
import time
t = time.time()
# do stuff
data = df['text']
result = []
for i in range(0,len(data)):
  # print(i)
  result.append([])
  for token in gensim.utils.simple_preprocess(data[i], deacc=True):
    token = porter.stem(token)
    # token = lancaster.stem(token)
    # print(token)
    if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
      result[i].append(lemmatize_text(token))
  result[i] = list(itertools.chain.from_iterable(result[i]))
elapsed = time.time() - t


In [ ]:
elapsed # tokenize, lemmetized & stemmed took 423 seconds


In [73]:
# import pickle
# with open('text_tokens.pkl', 'wb') as f:
#    pickle.dump(result, f)
# !cp text_tokens.pkl "./gdrive/MyDrive/Colab Datasets/UFOSighting_data/intermediate_data/"

open_file = open("./gdrive/MyDrive/Colab Datasets/UFOSighting_data/intermediate_data/text_tokens.pkl", "rb")
result = pickle.load(open_file)
open_file.close()


In [78]:
dictionary = gensim.corpora.Dictionary(result)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 babi
1 ball
2 befor
3 blue
4 bright
5 chang
6 clear
7 close
8 closer
9 cloud
10 color


In [79]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in result]
bow_corpus[0]

[(0, 1),
 (1, 1),
 (2, 2),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 2),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 2),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 5),
 (21, 4),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 1),
 (29, 3),
 (30, 1),
 (31, 4),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 1),
 (40, 1),
 (41, 1),
 (42, 1),
 (43, 1),
 (44, 1),
 (45, 4),
 (46, 1),
 (47, 1),
 (48, 1),
 (49, 1),
 (50, 1),
 (51, 2)]

### TF-IDF

In [82]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.17369070083509305),
 (1, 0.06900016857935845),
 (2, 0.09166547425724622),
 (3, 0.06035538212543421),
 (4, 0.028094390789480412),
 (5, 0.05980665172770389),
 (6, 0.05597160632749797),
 (7, 0.05759170773742988),
 (8, 0.14512335017423897),
 (9, 0.055916083066514764),
 (10, 0.04711326610159633),
 (11, 0.1496167297498618),
 (12, 0.12872115920524677),
 (13, 0.0367434667059634),
 (14, 0.05551675747407544),
 (15, 0.16480569958555688),
 (16, 0.09908125844780727),
 (17, 0.11883652274521352),
 (18, 0.16933950802016978),
 (19, 0.10442450066677902),
 (20, 0.44620727393716453),
 (21, 0.4876304765148966),
 (22, 0.09680366859367075),
 (23, 0.06418370708202457),
 (24, 0.064831373693197),
 (25, 0.06268945595214917),
 (26, 0.13955697130952113),
 (27, 0.11333090152295847),
 (28, 0.0912732437575585),
 (29, 0.11880604506893591),
 (30, 0.031037715012136407),
 (31, 0.08863353753823615),
 (32, 0.1418051857715567),
 (33, 0.09055876962085325),
 (34, 0.15680775113276732),
 (35, 0.21832888895820587),
 (36, 

### LDA model: to extract underlining topics from all sighting descriptions

In [83]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
# run time = 3m 12s

Topic: 0 
Words: 0.037*"note" + 0.036*"nuforc" + 0.034*"date" + 0.027*"sight" + 0.027*"approxim" + 0.025*"indic" + 0.019*"report" + 0.011*"shape" + 0.011*"time" + 0.009*"abov"
Topic: 1 
Words: 0.023*"orang" + 0.019*"like" + 0.016*"seen" + 0.015*"went" + 0.014*"object" + 0.014*"disappear" + 0.013*"bright" + 0.012*"minut" + 0.010*"watch" + 0.009*"start"
Topic: 2 
Words: 0.041*"object" + 0.029*"white" + 0.027*"color" + 0.024*"green" + 0.024*"blue" + 0.022*"bright" + 0.019*"chang" + 0.018*"shape" + 0.017*"video" + 0.016*"like"
Topic: 3 
Words: 0.045*"object" + 0.015*"appear" + 0.013*"observ" + 0.012*"aircraft" + 0.010*"sight" + 0.009*"altitud" + 0.009*"approxim" + 0.008*"travel" + 0.008*"speed" + 0.008*"time"
Topic: 4 
Words: 0.046*"craft" + 0.019*"drive" + 0.015*"white" + 0.015*"shape" + 0.013*"road" + 0.013*"triangl" + 0.012*"hover" + 0.011*"flash" + 0.010*"veri" + 0.010*"bright"
Topic: 5 
Words: 0.035*"star" + 0.030*"bright" + 0.017*"appear" + 0.016*"second" + 0.015*"like" + 0.015*"disa

Build LDA model using TF-IDF data

In [84]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.007*"object" + 0.005*"appear" + 0.005*"observ" + 0.004*"aircraft" + 0.004*"travel" + 0.004*"bright" + 0.004*"craft" + 0.004*"south" + 0.004*"north" + 0.004*"orang"
Topic: 1 Word: 0.028*"provid" + 0.027*"inform" + 0.026*"anonym" + 0.026*"elect" + 0.024*"contact" + 0.023*"remain" + 0.022*"total" + 0.022*"nuforc" + 0.021*"note" + 0.007*"orang"
Topic: 2 Word: 0.010*"orang" + 0.008*"bright" + 0.008*"disappear" + 0.007*"object" + 0.006*"travel" + 0.006*"star" + 0.006*"white" + 0.006*"east" + 0.006*"veri" + 0.006*"plane"
Topic: 3 Word: 0.041*"madar" + 0.039*"node" + 0.007*"green" + 0.007*"fall" + 0.005*"like" + 0.005*"bright" + 0.005*"blue" + 0.005*"star" + 0.004*"ball" + 0.004*"flash"
Topic: 4 Word: 0.008*"object" + 0.007*"shape" + 0.006*"cloud" + 0.006*"craft" + 0.005*"white" + 0.005*"silver" + 0.005*"hover" + 0.005*"like" + 0.005*"disappear" + 0.005*"cigar"
Topic: 5 Word: 0.011*"photo" + 0.008*"pictur" + 0.007*"object" + 0.007*"video" + 0.005*"camera" + 0.004*"appear" + 0.

Test the probablity for a section of text of being in each Topic:

In [81]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
                                               bow_doc_4310[i][1]))

Word 1 ("ball") appears 1 time.
Word 10 ("color") appears 1 time.
Word 24 ("glow") appears 4 time.
Word 29 ("night") appears 1 time.
Word 30 ("note") appears 1 time.
Word 40 ("seen") appears 1 time.
Word 41 ("shoot") appears 1 time.
Word 50 ("watch") appears 3 time.
Word 56 ("distanc") appears 1 time.
Word 61 ("left") appears 1 time.
Word 85 ("like") appears 2 time.
Word 86 ("line") appears 2 time.
Word 95 ("said") appears 1 time.
Word 110 ("fast") appears 2 time.
Word 113 ("know") appears 1 time.
Word 116 ("someth") appears 1 time.
Word 119 ("abov") appears 1 time.
Word 121 ("approxim") appears 1 time.
Word 128 ("date") appears 1 time.
Word 134 ("float") appears 1 time.
Word 136 ("gone") appears 1 time.
Word 140 ("indic") appears 1 time.
Word 141 ("insid") appears 1 time.
Word 147 ("nuforc") appears 1 time.
Word 148 ("orang") appears 1 time.
Word 149 ("outsid") appears 1 time.
Word 163 ("account") appears 1 time.
Word 167 ("area") appears 3 time.
Word 186 ("hover") appears 2 time.
Wor

In [95]:
for index, score in sorted(lda_model[bow_corpus[2]], key=lambda tup: -1*tup[1]):
    print(bow_corpus[2])
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))

[(2, 2), (13, 1), (29, 2), (61, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 2), (77, 1), (78, 1), (79, 1), (80, 1), (81, 4), (82, 1), (83, 1), (84, 1), (85, 2), (86, 2), (87, 1), (88, 1), (89, 2), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 2), (106, 1), (107, 1), (108, 2), (109, 2)]

Score: 0.7884594798088074	 
Topic: 0.018*"like" + 0.012*"time" + 0.010*"know" + 0.009*"thing" + 0.009*"night" + 0.009*"veri" + 0.009*"someth" + 0.008*"hous" + 0.007*"sound" + 0.007*"becaus"
[(2, 2), (13, 1), (29, 2), (61, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 2), (77, 1), (78, 1), (79, 1), (80, 1), (81, 4), (82, 1), (83, 1), (84, 1), (85, 2), (86, 2), (87, 1), (88, 1), (89, 2), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 2), (106, 1), (107, 1), (108, 2), (109, 2)]

Score: 0.1969919353723526	 
Topi

In [97]:
df['text'][2]

'I woke up late in the afternoon 3:30-4pm.  I went to have a bath, while shaving my legs i noticed indentations around my left ankel.  I then noticed that they went all the way around the backs of both my legs and up to my thighs.  It scared me because a few nights before I was telling my fiancee that was pretty sure that i was abducted before, he laughed then that was that.  I showed him the indentations and he was shocked.  He suffers from insomnia.  We woke up at the same time.  He only sleeps maybe 4-5 hrs a wk.  He said that he slept all night with me.  The indentations look like my legs were seperated into stirups, or like how a massage chair would hold your legs.  The indents were in rows of 1-1.5 lines from my ankles all the way up to my thighs just below my buttocks.  They were perfectly lined.  They disappeared within 2 hrs of waking up.  I couldnt talk a picture because i had just lost my cell phone.'

Bt evaluating the Score above (a.k.a. probabiltiy of having that topic given the text), we can observe that the text (shown above) has the highest probablity (0.78) of belong to topic:

Topic: 0.018*"like" + 0.012*"time" + 0.010*"know" + 0.009*"thing" + 0.009*"night" + 0.009*"veri" + 0.009*"someth" + 0.008*"hous" + 0.007*"sound" + 0.007*"becaus"

Apparently, more interpretation of each topic is needed. In this given example, we can see this topic involves less of observing a typical light in the sky but more so on knowing or seeing something happening to the person herself, comparing to the example shown below:

In [107]:
for index, score in sorted(lda_model[bow_corpus[5]], key=lambda tup: -1*tup[1]):
    print(bow_corpus[5])
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))

[(3, 1), (10, 2), (13, 2), (25, 1), (30, 1), (31, 15), (40, 2), (65, 2), (85, 1), (89, 2), (106, 2), (113, 1), (120, 1), (147, 1), (156, 1), (163, 1), (164, 2), (165, 1), (166, 1), (167, 1), (168, 1), (169, 1), (170, 2), (171, 1), (172, 1), (173, 1), (174, 1), (175, 1), (176, 1), (177, 1), (178, 1), (179, 1), (180, 1), (181, 1), (182, 1), (183, 2), (184, 1), (185, 1), (186, 2), (187, 1), (188, 4), (189, 1), (190, 1), (191, 1), (192, 1), (193, 1), (194, 2), (195, 2), (196, 2), (197, 1), (198, 1), (199, 3), (200, 1), (201, 1), (202, 1), (203, 1), (204, 1), (205, 1), (206, 4), (207, 1), (208, 1), (209, 1), (210, 2), (211, 1), (212, 1), (213, 1), (214, 1), (215, 1)]

Score: 0.4810084104537964	 
Topic: 0.041*"object" + 0.029*"white" + 0.027*"color" + 0.024*"green" + 0.024*"blue" + 0.022*"bright" + 0.019*"chang" + 0.018*"shape" + 0.017*"video" + 0.016*"like"
[(3, 1), (10, 2), (13, 2), (25, 1), (30, 1), (31, 15), (40, 2), (65, 2), (85, 1), (89, 2), (106, 2), (113, 1), (120, 1), (147, 1), (156

In [108]:
df['text'][5]

"The object has flashing lights that are green, blue, red, and white.  The lights revolves around the object.  The object hovers in the same area for a long period of time. ((ADDENDUM FROM WITNESS)) Thank you for responding.  The object seemed sphere or disc shaped.  The object was extremely far away, but we were able to see every color.  At one point a plane flew under the object and we were able to see how large it was in comparison.  It was at least two or three times the size of the aircraft.  The object ended up hovering for over an hour that we saw.  At one point the object dipped down and we noticed a smaller object below it that was similar.  When the larger object reached the smaller one, all of the sudden the smaller object disappeared.  We then noticed a large plane that flew under the large object flying North. The large object then suddenly disappeared. Please understand that I have never believed in UFO's nor do I believe in conspiracy theories.  This is a true account of

Bt evaluating the Score above (a.k.a. probabiltiy of having that topic given the text), we can observe that the text (shown above) has the highest probablity (0.48) of belong to topic:

Topic: 0.041*"object" + 0.029*"white" + 0.027*"color" + 0.024*"green" + 0.024*"blue" + 0.022*"bright" + 0.019*"chang" + 0.018*"shape" + 0.017*"video" + 0.016*"like"

which indicates that this passage is likely to describe a bright object that the reporter observed in the sky. 

### Conclusion & Future Work
By building a LDA model that extract possible underlying topics using the text description provided in the data, there are many useful insights & functionalities that I can draw. 

## Scratch Notes

In [22]:
# list(gensim.utils.simple_preprocess("i you he she I it we you they", deacc=True))
# gensim.parsing.preprocessing.STOPWORDS
test = 'My wife was driving southeast on a fairly populated main side road, it was dark out side at about 6:43pm, And my wife exclaimed” falling star baby look quick!” When I looked up I saw not a falling star but a bright ball of light , one that was closer than any shooting star I have ever seen, it had a blue glow  changing into green colors of light as it fell from the sky. It fell as if falling from an invisible opening in the sky... the night was a crystal clear night sky  so no clouds or precipitation to obstruct our view and the object was closer than any I have ever witnessed before. The way the object fell was too slow to be a meteor or falling star, also noting that there was no “light trail” following behind the object as it fell. We watched it fall for about 5 seconds before it disappeared into the dark night sky somewhere close to the earth surface.  It was the strangest light; object; ufo... I have encountered and felt a duty to report it.'
result = []
for token in gensim.utils.simple_preprocess(test, deacc=True):
  # token = porter.stem(token)
  token = lancaster.stem(token)
  if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
    # print(lemmatize_text(token))
    result.append(lemmatize_text(token))
result2 = []
for word in result:   #for each word in line.split()
    if word not in result2:    #if a word isn't in line.split            
        result2.append(word)
# result[0:5]
print(len(result))
print(len(result2))
print(result2)

57
42
[['driv'], ['southeast'], ['fair'], ['main'], ['road'], ['dark'], ['exclaim'], ['star'], ['baby'], ['look'], ['quick'], ['bright'], ['light'], ['clos'], ['shoot'], ['seen'], ['glow'], ['chang'], ['green'], ['invis'], ['night'], ['cryst'], ['clear'], ['cloud'], ['precipit'], ['obstruct'], ['view'], ['object'], ['slow'], ['ther'], ['trail'], ['follow'], ['watch'], ['second'], ['disappear'], ['somewh'], ['surfac'], ['strangest'], ['encount'], ['felt'], ['duty'], ['report']]
